In [ ]:
import ee
from ltm.data import _get_roi_scale_crs, _initialize_ee
import requests
import os
import shutil


def save_image(image, scale, crs, region, file_path):
    download_params = {
        "scale": scale,
        "crs": crs,
        "region": region,
        "format": "GEO_TIFF",
    }

    url = image.getDownloadURL(download_params)

    response = requests.get(url, stream=True)
    with open(file_path, "wb") as out_file:
        shutil.copyfileobj(response.raw, out_file)


def reduce(reducer):
    _initialize_ee()

    image = ee.Image("projects/leaf-type-mixture/assets/DEM")
    reprojected = image.reduceResolution(reducer=getattr(ee.Reducer, reducer)(), maxPixels=65536)

    roi, scale, crs = _get_roi_scale_crs("../data/processed/data.tif")
    
    save_image(reprojected, scale, crs, roi, f"../data/processed/DEM_{reducer}.tif")

reduce("mean")
reduce("median")
reduce("mode")
reduce("max")
reduce("min")